In [1]:

# df= pd.read_csv("train_set1.csv")
import pandas as pd 
import string
import re
import pandas as pd 
import os
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import numpy as np
import random
import math
import time
from torch.nn.utils.rnn import pad_sequence

In [2]:
df_train = pd.read_csv('/content/drive/MyDrive/df_train2.csv')
df_valid = pd.read_csv('/content/drive/MyDrive/df_valid2.csv')
df_test = pd.read_csv('/content/drive/MyDrive/df_test2.csv')


In [4]:
df_train=df_train.drop(['Unnamed: 0'],axis = 1)
df_valid=df_valid.drop(['Unnamed: 0'],axis = 1)
df_test=df_test.drop(['Unnamed: 0'],axis = 1)


In [5]:
#convert string to list
punctuation_string = string.punctuation
for i, row in df_train.iterrows():
    word1 = row['code_tokens']
    word2 = row['docstring_tokens']
    for j in punctuation_string:
        word1 = word1.replace(j, '')
        word2 = word2.replace(j, '')
    word_list1 = word1.split()
    word_list2 = word2.split()
    row['code_tokens'] = word_list1
    row['docstring_tokens'] = word_list2

In [6]:
punctuation_string = string.punctuation
for i, row in df_valid.iterrows():
    word1 = row['code_tokens']
    word2 = row['docstring_tokens']
    for j in punctuation_string:
        word1 = word1.replace(j, '')
        word2 = word2.replace(j, '')
    word_list1 = word1.split()
    word_list2 = word2.split()
    row['code_tokens'] = word_list1
    row['docstring_tokens'] = word_list2

In [7]:
punctuation_string = string.punctuation
for i, row in df_test.iterrows():
    word1 = row['code_tokens']
    word2 = row['docstring_tokens']
    for j in punctuation_string:
        word1 = word1.replace(j, '')
        word2 = word2.replace(j, '')
    word_list1 = word1.split()
    word_list2 = word2.split()
    row['code_tokens'] = word_list1
    row['docstring_tokens'] = word_list2

In [8]:
def lengthCheck(df):
    list1=[]
    list2=[]
    for i, row in df.iterrows():
        length1 = len(df.iloc[i].iat[0])
        length2 = len(df.iloc[i].iat[1])
        list1.append(length1)
        list2.append(length2)
    return list1,list2

In [9]:
length_list1,length_list2 = lengthCheck(df_train)
length_list3,length_list4 = lengthCheck(df_valid)


In [10]:
num = sum(i <200 for i in length_list1)
print(num)
print(len(length_list1))
num = sum(i <200 for i in length_list2)
print(num)
print(len(length_list2))

18402
20000
19970
20000


In [11]:
def cutLength(df):
    for i, row in df.iterrows():
        length1 = len(df.iloc[i].iat[0])
        length2 = len(df.iloc[i].iat[1])
        a = df.iloc[i].iat[0]
        b = df.iloc[i].iat[1]
        if length1>200:
            del a[200:]
        if length2>200:
             del b[200:]
    return df

In [12]:
df_train_cut = cutLength(df_train)
#df_valid_cut = cutLength(df_valid)

In [13]:
df_valid_cut = cutLength(df_valid)

In [14]:
df_test_cut = cutLength(df_test)

In [15]:
list_x = []
list_Y = []
for i, row in df_train_cut.iterrows():
    list_x.append(row['code_tokens'])
    list_Y.append(row['docstring_tokens'])
list_a = [m for item in list_x for m in item]
list_b = [m for item in list_Y for m in item]

In [16]:
print(len(list_a))

1396076


In [17]:
from collections import Counter
count = Counter(list_a)
word_a = [x[0] for x in count.items() if x[1] > 3]
print(len(word_a))
count1 = Counter(list_b)
word_b = [x[0] for x in count1.items() if x[1] > 3]
print(len(word_b))

10855
4250


In [18]:
size1 = len(word_a)
size2 = len(word_b)
tokenids = range(3,size1+3)
docids = range(3,size2+3)
src_dict = dict(zip(word_a, tokenids))
trg_dict = dict(zip(word_b, docids))
src_dict['<sos>']=0
src_dict['<eos>']=1
src_dict['<unk>']=2
trg_dict['<sos>']=0
trg_dict['<eos>']=1
trg_dict['<unk>']=2

In [19]:
list_src = df_train_cut['code_tokens'].values.tolist()
list_trg = df_train_cut['docstring_tokens'].values.tolist()

In [20]:
valid_src = df_valid_cut['code_tokens'].values.tolist()
valid_trg = df_valid_cut['docstring_tokens'].values.tolist()

In [21]:
test_src = df_test_cut['code_tokens'].values.tolist()
test_trg = df_test_cut['docstring_tokens'].values.tolist()

In [22]:
#Mapping data through the dictionary
dic_src = []
dic_trg=[]
for i in list_src:
    list_vocab_a=[]
    list_vocab_a.append(src_dict['<sos>'])
    for m in i:
        if not m in src_dict.keys():
            list_vocab_a.append(src_dict['<unk>'])
        else:
            list_vocab_a.append(src_dict[m])
    list_vocab_a.append(src_dict['<eos>'])
    dic_src.append(list_vocab_a)
for i in list_trg:
    list_vocab_a=[]
    list_vocab_a.append(src_dict['<sos>'])
    for m in i:
        if not m in trg_dict.keys():
            list_vocab_a.append(trg_dict['<unk>'])
        else:
            list_vocab_a.append(trg_dict[m])
    list_vocab_a.append(trg_dict['<eos>'])
    dic_trg.append(list_vocab_a)

In [23]:
test_src = []
test_trg=[]
for i in valid_src:
    list_vocab_a=[]
    list_vocab_a.append(src_dict['<sos>'])
    for m in i:
        if not m in src_dict.keys():
            list_vocab_a.append(src_dict['<unk>'])
        else:
            list_vocab_a.append(src_dict[m])
    list_vocab_a.append(src_dict['<eos>'])
    test_src.append(list_vocab_a)
for i in valid_trg:
    list_vocab_a=[]
    list_vocab_a.append(trg_dict['<sos>'])
    for m in i:
        if not m in trg_dict.keys():
            list_vocab_a.append(trg_dict['<unk>'])
        else:
            list_vocab_a.append(trg_dict[m])
    list_vocab_a.append(trg_dict['<eos>'])
    test_trg.append(list_vocab_a)

In [24]:
testData_src = []
testData_trg=[]
for i in test_src:
    list_vocab_a=[]
    list_vocab_a.append(src_dict['<sos>'])
    for m in i:
        if not m in src_dict.keys():
            list_vocab_a.append(src_dict['<unk>'])
        else:
            list_vocab_a.append(src_dict[m])
    list_vocab_a.append(src_dict['<eos>'])
    testData_src.append(list_vocab_a)
for i in test_trg:
    list_vocab_a=[]
    list_vocab_a.append(trg_dict['<sos>'])
    for m in i:
        if not m in trg_dict.keys():
            list_vocab_a.append(trg_dict['<unk>'])
        else:
            list_vocab_a.append(trg_dict[m])
    list_vocab_a.append(trg_dict['<eos>'])
    testData_trg.append(list_vocab_a)

In [25]:
print(len(src_dict),len(trg_dict))


10858 4253


In [26]:
trg_dict['ignore_idx']=len(trg_dict)

In [27]:
class Data_Subset(Dataset):

    def __init__(self,s1,s2,ignore):
        self.s1 = s1
        self.s2 = s2
        self.src = s1
        self.trg = s2
        self.ig = ignore
    def __getitem__(self, index):
  
        
        while len(self.src[index])< 202:
            self.src[index].append(1)
        
        while len(self.trg[index])< 202:
            self.trg[index].append(self.ig)
        return torch.LongTensor(self.src[index]), torch.LongTensor(self.trg[index])
    def __len__(self):
        return len(self.s1)
    
dataset_train = Data_Subset(dic_src, dic_trg,trg_dict['ignore_idx'])

In [28]:
dataset_valid = Data_Subset(test_src, test_trg,trg_dict['ignore_idx'])

In [29]:
dataset_test = Data_Subset(testData_src, testData_src,trg_dict['ignore_idx'])

In [30]:
train_loader = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=36,  
        shuffle=True,
        num_workers=0, 
    )

In [31]:
print(train_loader)

In [32]:
valid_loader = torch.utils.data.DataLoader(
        dataset_valid,
        batch_size=36,  
        shuffle=True,
        num_workers=0, 
    )

In [33]:
test_loader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=36,  
        shuffle=True,
        num_workers=0, 
    )

In [34]:
train_iter = iter(valid_loader)
a,b = train_iter.next()
print(a.shape)
print(b.shape)

torch.Size([36, 202])
torch.Size([36, 202])


In [35]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        return hidden, cell


In [36]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        
        
        return prediction, hidden, cell

In [37]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output   
            teacher_force = random.random() < teacher_forcing_ratio    
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1
        
        return outputs

In [38]:
INPUT_DIM = len(src_dict)
OUTPUT_DIM = len(trg_dict)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)


In [39]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10858, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4254, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=4254, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [40]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss(ignore_index = trg_dict['ignore_idx'])

In [41]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, (src,trg) in enumerate(iterator):
        
        optimizer.zero_grad()
        
        output = model(src.cuda(), trg.cuda())
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg.cuda())
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [42]:
#define a evaluate class to calculate the valid loss
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, (src,trg) in enumerate(iterator):

            output = model(src.cuda(), trg.cuda(), 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg.cuda())
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [43]:
train_losses=[]
valid_losses = []
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS): 
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    print(train_loss,valid_loss)

5.650188896295836 5.437318147659302
5.438117491255562 5.329034534454346
5.373317169628555 5.3123935546875
5.356555527062725 5.304422233581543
5.339817428760392 5.290379026412964
5.331883860148972 5.293916421890259
5.320893073253495 5.29627812576294
5.316752675625918 5.29040175819397
5.3068804131995 5.29250030708313
5.302488488259075 5.296533763885498
